### stage 1, setting up
documentation available in docs/docs.ipynb

In [ ]:
import duckdb

In [ ]:
%run "helpers.ipynb"

In [ ]:
%run "tests/unit_tests.ipynb"

### stage 2, ingest raw data and stage in bronze

In [ ]:
con = duckdb.connect(database = ':memory:', read_only=False)

In [ ]:
#Initial load datasets into duckdb
dataset_loader(connection=con)

In [ ]:
#stage the raw files in the bronze layer (data/output/bronze)
dataset_stager(connection=con, medallion_zone='bronze', datasets=[dataset.dataset_name for dataset in dataset_info])

### stage 3, clean the raw files and stage in silver

In [ ]:
#clean the airbnb data
#creates in memory table 'airbnb_cleaned'
clean_airbnb_for_silver(connection=con)


In [ ]:
#clean the rental data
#creates in memory table 'rentals_cleaned'
clean_rentals_for_silver(connection=con)

In [ ]:
#Save the cleaned datasets in the silver layer
dataset_stager(connection=con, medallion_zone='silver', datasets=['airbnb_cleaned', 'rentals_cleaned'])

### stage 4, aggregate both datasets and stage in gold

In [ ]:
#combine the rental and airbnb datasets and aggregate (average rental and airbnb price per postcode)
#creates in memory table 'aggregate_data'
aggregate_data(connection=con)

In [ ]:
#save the agregated dataset in the gold layer
dataset_stager(connection=con, medallion_zone='gold', datasets=['aggregate_data'])

In [ ]:
#Top 10 zipcodes with on average highest airbnb prices based on X observations
con.sql("""
SELECT zipcode
, average_price_airbnb
, airbnb_count AS observation_count 
FROM aggregate_data 
ORDER BY average_price_airbnb desc 
LIMIT 10"""
).show()

In [ ]:
#Top 10 zipcodes with on average highest rental prices based on X observations
con.sql("""SELECT zipcode
,average_price_rental
,rental_count AS observation_count 
FROM aggregate_data 
ORDER BY average_price_rental desc LIMIT 10
""").show()

In [ ]:
con.close()